補充>
Data augmentation:
一張圖片經過旋轉、調整大小、比例尺寸，或者改變亮度色溫、翻轉等處理後，
我們人眼仍能辨識出來是相同的相片，但是對機器來說那可是完全不同的新圖像了，
因此， Data augmentation就是將dataset中既有的圖片予以修改變形，
以創造出更多的圖片來讓機器學習，彌補資料量不足的困擾。

In [1]:
import cv2
import numpy as np
img_path = 'lena.png'
img = cv2.imread(img_path, cv2.IMREAD_COLOR)
type(img)

numpy.ndarray

# 改變飽和度

In [9]:
# 為了要改變飽和度，我們先把 color space 轉成 HSV 格式
img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
change_percentage = 0.2


# 針對飽和度的值做改變，超過界線 0~1 的都會 bound
# 在 HSV color space 減少飽和度
img_hsv_down=img_hsv.astype('float32')
img_hsv_down[...,1]= img_hsv[...,1]/255-change_percentage
#check the boundary problem
img_hsv_down[img_hsv_down[...,1]<0]=0
img_hsv_down[...,1]=img_hsv_down[...,1]*255
img_hsv_down=img_hsv_down.astype('uint8')

# 在 HSV color space 增加飽和度
img_hsv_up=img_hsv.astype('float32')
img_hsv_up[...,1]= img_hsv[...,1]/255+change_percentage
#check the boundary problem
img_hsv_up[img_hsv_up[...,1]>1]=1
img_hsv_up[...,1]=img_hsv_up[...,1]*255
img_hsv_up=img_hsv_up.astype('uint8')

# 轉換 color space 回 BGR
img_hsv_down = cv2.cvtColor(img_hsv_down, cv2.COLOR_HSV2BGR)
img_hsv_up = cv2.cvtColor(img_hsv_up, cv2.COLOR_HSV2BGR)



# 組合圖片 + 顯示圖片
img_hsv_change = np.hstack((img, img_hsv_down, img_hsv_up)) #沿着水平方向将数组堆叠起来。
while True:
    cv2.imshow('change saturation', img_hsv_change)
    
    k = cv2.waitKey(0)
    if k == 27:
        cv2.destroyAllWindows()
        break

# 直方圖均衡

In [5]:
# case 1
# 每個 channel 個別做直方圖均衡
B,G,R = cv2.split(img)
B_eq=cv2.equalizeHist(B)
G_eq=cv2.equalizeHist(G)
R_eq=cv2.equalizeHist(R)
equalHist_by_channel = cv2.merge([B_eq,G_eq,R_eq])

# 組合經過直方圖均衡的每個 channel
img_bgr_equal = np.hstack((img,equalHist_by_channel))


# case 2 - 轉換 color space 後只對其中一個 channel 做直方圖均衡
img_hsv_equal = cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
img_split_hsv_equal = cv2.split(img_hsv_equal) 
#只對 V channel做直方 
img_split_hsv_equal[2] = cv2.equalizeHist(img_split_hsv_equal[2])
img_hsv_equal = cv2.merge(img_split_hsv_equal)

# 組合圖片 + 顯示圖片
img_bgr_equalHist = np.hstack((img, img_bgr_equal, img_hsv_equal))
while True:
    # 比較 (原圖, BGR color space 對每個 channel 做直方圖均衡, HSV color space 對明度做直方圖均衡)
    cv2.imshow('bgr equal histogram', img_bgr_equalHist)
    k = cv2.waitKey(0)
    if k == 27:
        cv2.destroyAllWindows()
        break


# 調整對比 / 明亮

In [8]:
# alpha: 控制對比度 (1.0~3.0)
# beta: 控制明亮度 (0~255)
add_contrast = cv2.convertScaleAbs(img, alpha=2.0, beta=0)
add_lighness = cv2.convertScaleAbs(img, alpha=1.0, beta=120)

# 組合圖片 + 顯示圖片
img_contrast_light = np.hstack((img, add_contrast, add_lighness))
while True:
    # 比較不同程度的對比 / 明亮
    cv2.imshow('adjust contrast and brighness', img_contrast_light)
    k = cv2.waitKey(0)
    if k == 27:
        cv2.destroyAllWindows()
        break